# Full notebook
This notebook has the whole process put together:
1. Loading dataset
2. Defining grammar
3. Searching through the grammar and evaluating candidate pipelines

## 0. Imports

In [93]:
# uncomment the following if not all packages are added.

# import Pkg
# using Pkg
# Pkg.add("HTTP")
# Pkg.add("JSON")
# Pkg.add("DataFrames")
# Pkg.add("OpenML")
# Pkg.add("DataFrames") 
# Pkg.add("CSV") 
# Pkg.add("Suppressor")
# Pkg.add("StatsBase")

In [94]:
using ScikitLearn
using ScikitLearn.Pipelines: Pipeline, FeatureUnion
using ScikitLearn.CrossValidation: cross_val_score
using XGBoost
using Revise
using Random
using Statistics: mean
using ExprRules: get_executable
using Suppressor
using Random
using Dates

include("./Herb.jl/src/Herb.jl")
include("./Herb.jl/HerbGrammar.jl/src/HerbGrammar.jl")
include("./Herb.jl/HerbData.jl/src/HerbData.jl")
include("./Herb.jl/HerbEvaluation.jl/src/HerbEvaluation.jl")
include("./Herb.jl/HerbConstraints.jl/src/HerbConstraints.jl")
include("./Herb.jl/HerbSearch.jl/src/HerbSearch.jl")
include("helper.jl")

get_class_type_dataset (generic function with 1 method)

In [95]:
# import the sk-learn functions
@sk_import decomposition: (PCA)
@sk_import preprocessing: (StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler, Binarizer, PolynomialFeatures)
@sk_import feature_selection: (VarianceThreshold, SelectKBest, SelectPercentile, SelectFwe, RFE)
@sk_import tree: (DecisionTreeClassifier)
@sk_import ensemble: (RandomForestClassifier, GradientBoostingClassifier)
@sk_import linear_model: (LogisticRegression)
@sk_import neighbors: (NearestNeighbors)
@sk_import svm: (LinearSVC)

PyObject <class 'sklearn.svm._classes.LinearSVC'>

## 1. Loading datasets

In [96]:
# Iris: 61, Seeds: 1499, Blood transfusion: 1464, Monks: 334, Diabetes: 37, ilpd: 1480

# load dataset
# dataset = get_dataset(61)

# it does not work for the seeds table dataset!
dataset = get_dataset(1499)

# does not work either on dataset 1464!

# shuffle the dataset
dataset_shuffled = dataset[shuffle(1:end), :]

# split into train and test sets (90:10)
split_index = floor(Int, size(dataset_shuffled, 1) * 0.90)
train_df = dataset_shuffled[1:split_index, :]
test_df = dataset_shuffled[split_index+1:end, :]

# show first five entries
first(dataset_shuffled, 5)

Row,V1,V2,V3,V4,V5,V6,V7,Class
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Cat…
1,19.38,16.72,0.8716,6.303,3.791,3.678,5.965,2
2,16.82,15.51,0.8786,6.017,3.486,4.004,5.841,2
3,11.23,12.82,0.8594,5.089,2.821,7.524,4.957,3
4,12.37,13.47,0.8567,5.204,2.96,3.919,5.001,3
5,12.55,13.57,0.8558,5.333,2.968,4.419,5.176,3


In [97]:
# show metadata
print("size: ", size(dataset_shuffled))
describe(dataset_shuffled)

size: (210, 8)

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Union…,Any,Union…,Any,Int64,DataType
1,V1,14.8475,10.59,14.355,21.18,0,Float64
2,V2,14.5593,12.41,14.32,17.25,0,Float64
3,V3,0.870999,0.8081,0.87345,0.9183,0,Float64
4,V4,5.62853,4.899,5.5235,6.675,0,Float64
5,V5,3.2586,2.63,3.237,4.033,0,Float64
6,V6,3.7002,0.7651,3.599,8.456,0,Float64
7,V7,5.40807,4.519,5.223,6.55,0,Float64
8,Class,,1,,3,0,"CategoricalValue{String, UInt32}"


In [98]:
# split into features and labels
train_X = train_df[:, 1:end-1]
train_Y = train_df[:, end]
test_X = test_df[:, 1:end-1]
test_Y = test_df[:, end]

# print ratio train/test
ratio = trunc(Int, 10.0 * (size(train_df)[1] / (size(train_df)[1] + size(test_df)[1])))
print("train:test ratio = ", ratio , ":", (10-ratio))

train:test ratio = 9:1

## 2. Defining grammar

In [99]:
grammar = Herb.HerbGrammar.@cfgrammar begin

    # this is the version with multiple classifiers possible
    # START   = CLASSIF | sequence(PRE, CLASSIF)
    # PRE     = PREPROC | FSELECT | sequence(PRE, PRE) | parallel(BRANCH, BRANCH)
    # BRANCH  = PRE | CLASSIF | sequence(PRE, CLASSIF) 

    # this is the version with only one classifier
    START   = Pipeline([CLASSIF]) | Pipeline([PRE, CLASSIF])
    PRE     = PREPROC | FSELECT | ("seq", Pipeline([PRE, PRE]))  | ("par", FeatureUnion([PRE, PRE])) 

    # preprocessing functions
    PREPROC =   
        ("StandardScaler" * string(rand(Int)), StandardScaler()) |
        ("RobustScaler", RobustScaler()) |
        ("MinMaxScaler", MinMaxScaler()) |
        ("MaxAbsScaler", MaxAbsScaler()) |
        ("PCA", PCA()) |
        ("Binarizer", Binarizer()) |
        ("PolynomialFeatures", PolynomialFeatures())

    # feature selection functions
    FSELECT =  
        ("VarianceThreshold", VarianceThreshold()) |
        # ("SelectKBest",  SelectKBest()) |
        ("SelectPercentile",  SelectPercentile()) |
        ("SelectFwe",  SelectFwe()) |
        ("Recursive Feature Elimination",  RFE(LinearSVC())) 

    # classifiers
    CLASSIF =
        ("DecisionTree", DecisionTreeClassifier()) |
        ("RandomForest", RandomForestClassifier()) |
        ("Gradient Boosting Classifier", GradientBoostingClassifier()) |
        ("LogisticRegression", LogisticRegression()) |
        ("NearestNeighborClassifier", NearestNeighbors())
end


1: START = Pipeline([CLASSIF])
2: START = Pipeline([PRE, CLASSIF])
3: PRE = PREPROC
4: PRE = FSELECT
5: PRE = ("seq", Pipeline([PRE, PRE]))
6: PRE = ("par", FeatureUnion([PRE, PRE]))
7: PREPROC = ("StandardScaler" * string(rand(Int)), StandardScaler())
8: PREPROC = ("RobustScaler", RobustScaler())
9: PREPROC = ("MinMaxScaler", MinMaxScaler())
10: PREPROC = ("MaxAbsScaler", MaxAbsScaler())
11: PREPROC = ("PCA", PCA())
12: PREPROC = ("Binarizer", Binarizer())
13: PREPROC = ("PolynomialFeatures", PolynomialFeatures())
14: FSELECT = ("VarianceThreshold", VarianceThreshold())
15: FSELECT = ("SelectPercentile", SelectPercentile())
16: FSELECT = ("SelectFwe", SelectFwe())
17: FSELECT = ("Recursive Feature Elimination", RFE(LinearSVC()))
18: CLASSIF = ("DecisionTree", DecisionTreeClassifier())
19: CLASSIF = ("RandomForest", RandomForestClassifier())
20: CLASSIF = ("Gradient Boosting Classifier", GradientBoostingClassifier())
21: CLASSIF = ("LogisticRegression", LogisticRegression())
22: CLASSI

In [100]:
function insert_name_indexes(p)
    p_start = ""
    i = 1
    while i <= 100
        try
            p = replace(p, """",""" => string(i)*"""",""", count=1)
            p_split = split(p, string(i) * """", """)
            p_start *= p_split[1] * string(i) * """", """
            p = p_split[2]
            i += 1
        catch
            break
        end
    end
    return split(p_start, string(i))[1]
end

insert_name_indexes (generic function with 1 method)

In [101]:
# all pipelines that can be assembled in x steps
cfe = Herb.HerbSearch.ContextFreeEnumerator(grammar, 4, :START)

# Find start program
start_program = nothing
c = 0
i = abs(rand(Int) % 1000)
for pipeline in cfe
    if c == i
        start_program = pipeline
        println(Herb.HerbSearch.rulenode2expr(pipeline, grammar))
        println(start_program)
        break
    end
    c = c + 1
end
print(i)

UndefVarError: UndefVarError: ContextFreeEnumerator not defined

In [102]:
function mh(grammar, enumeration_depth)
    current_program = start_program
    current_accuracy = -0.1
    i = 0
    not_improved_counter = 0
    max_seconds = 30
    max_time = Dates.Millisecond(max_seconds * 1000)
    t_start = now()
    while i < 100
        t_now = now()
        if t_now - t_start > max_time
            break
        end
        println("iteration: ", i)
        previous_accuracy = current_accuracy
        current_program, current_accuracy = one_iter_mh(current_program, grammar, enumeration_depth, t_start, max_time, current_accuracy)

        if current_accuracy == previous_accuracy
            not_improved_counter += 1
            if not_improved_counter == 10
                println("stopping because hasn't inproved in 10 iterations")
                break
            end
        else
            not_improved_counter = 0
        end
        if current_accuracy == 1.0
            println("stopping because accuracy is 1.0")
            break
        end
        i += 1
    end

    current_cost = 1.0 - current_accuracy

    println("final program: ", current_program)
    println("final cost: ", current_cost)
    return current_program, current_cost
end

mh (generic function with 1 method)

In [103]:
function one_iter_mh(current_program, grammar, enumeration_depth, t_start, max_time, current_program_accuracy)
    # println("Start vlns iteration")
    # println("current_program = ", current_program)
    # println("current_cost = ", pipeline_cost_function(eval(Herb.HerbSearch.rulenode2expr(current_program, grammar)), train_X, train_Y, test_X, test_Y))
    # 1. Construct neighbourhood
    neighbourhood_node_loc, dict = Herb.HerbSearch.constructNeighbourhoodRuleSubset(current_program, grammar)
    replacement_expression = Herb.HerbSearch.random_fill_propose(current_program, 
                                                                    neighbourhood_node_loc, 
                                                                    grammar,
                                                                    8, # max_depth = max depth of pipeline, depth of subprogram is bound by this
                                                                    dict)[1]
    original_program = deepcopy(current_program)  
    alternative_program = current_program                                                                                          
    if neighbourhood_node_loc.i == 0
        alternative_program = replacement_expression
    else
        neighbourhood_node_loc.parent.children[neighbourhood_node_loc.i] = replacement_expression
    end

    pipeline = eval(Meta.parse(insert_name_indexes(string(Herb.HerbSearch.rulenode2expr(alternative_program, grammar)))))
    alternative_program_cost = pipeline_cost_function(pipeline, train_X, train_Y, test_X, test_Y)
    alternative_program_accuracy = 1.0 - alternative_program_cost

    if (alternative_program_accuracy / current_program_accuracy) > ((rand(Int)%100000)/100000)
        return alternative_program, alternative_program_accuracy
    else
        return original_program, current_program_accuracy
    end
end

one_iter_mh (generic function with 1 method)

In [104]:
mh(grammar, 4) # enumeration_depth = max depth of subprogram that is being replaced, regardless of the NodeLoc

iteration: 0


MethodError: MethodError: no method matching constructNeighbourhoodRuleSubset(::Main.Herb.HerbGrammar.RuleNode, ::Main.Herb.HerbGrammar.ContextFreeGrammar)
Closest candidates are:
  constructNeighbourhoodRuleSubset(!Matched::Main.Herb.HerbGrammar.RuleNode, ::Main.Herb.HerbGrammar.ContextFreeGrammar) at c:\Users\aukes\Documents\Studie\Y3\BEP\research_project\Herb.jl\HerbSearch.jl\src\stocastic_functions\neighbourhood.jl:29

## 3. Search

In [42]:
"""
Fits the pipeline to the training set and measures the accuracy on test set.
input:  pipeline, train_X, train_Y, test_X, test_Y
output: accuracy of pipeline
"""
function evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)

    # # this gives the following warning often, so it is suppressed for now.
    # # ConvergenceWarning: lbfgs failed to converge
    @suppress_err begin
        try
            # fit the pipeline
            # print(pipeline)
            # print(" - ")
            model = ScikitLearn.fit!(pipeline, Matrix(train_X), Array(train_Y))

            # make predictions
            predictions = ScikitLearn.predict(model, Matrix(test_X))

            # measure the accuracy
            accuracy = mean(predictions .== test_Y)
            return accuracy
        catch e
            # println("Caught error [in evaluate_pipeline()]: ", e)
            return 0.0
        end
    end
end

evaluate_pipeline

In [43]:
"""Trains the pipeline and returns 1-accuracy"""
function pipeline_cost_function(pipeline, train_X, train_Y, test_X, test_Y)
    return 1.0 - evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)
end

pipeline_cost_function

In [ ]:
"""This function enumerates the grammar and finds the best pipeline. """
function find_best_pipeline_with_bfs_search(grammar, train_X, train_Y, test_X, test_Y, search_depth)
    best_accuracy = -1.0
    best_pipeline = nothing

    # enumerate the gramamar
    enumerator = Herb.HerbSearch.ContextFreeEnumerator(grammar, search_depth, :START)
    for rule in enumerator
        try
            # get pipeline and calculate accuracy
            pipeline = eval(Herb.HerbSearch.rulenode2expr(rule, grammar))
            accuracy = evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)

            # update best pipeline
            if (accuracy > best_accuracy) 
                best_accuracy = accuracy
                best_pipeline = pipeline
            end
            
            # print accuracy of pipeline
            print("\n accuracy: ", round(accuracy, digits=2), " by ", string(pipeline))
        catch
            continue
        end
    end
    return (best_accuracy, best_pipeline)
end

In [ ]:
# find the best pipeline and accuracy with depth 2
(best_accuracy, best_pipeline) = find_best_pipeline_with_bfs_search(grammar, train_X, train_Y, test_X, test_Y, 2)

# print result
println("\n\nBest pipeline: ", round(best_accuracy, digits=2))
print(best_pipeline)

In [ ]:
test_pipeline = Pipeline([("DecisionTree", DecisionTreeClassifier())])
pipeline_cost_function(test_pipeline, train_X, train_Y, test_X, test_Y)

In [ ]:
"""This function enumerates the grammar and finds the best pipeline. """
function find_best_pipeline_with_vlsn(grammar, train_X, train_Y, test_X, test_Y, search_depth)
    best_accuracy = -1.0
    best_pipeline = nothing

    # enumerate the gramamar
    # enumerator = Herb.HerbSearch.ContextFreeEnumerator(grammar, search_depth, :START)
    enumerator = Herb.HerbSearch.get_vlsn_enumerator(grammar, [], search_depth, :START, pipeline_cost_function)
    for rule in enumerator
        try
            # get pipeline and calculate accuracy
            pipeline = eval(Herb.HerbSearch.rulenode2expr(rule, grammar))
            accuracy = evaluate_pipeline(pipeline, train_X, train_Y, test_X, test_Y)

            # update best pipeline
            if (accuracy > best_accuracy) 
                best_accuracy = accuracy
                best_pipeline = pipeline
            end
            
            # print accuracy of pipeline
            print("\n accuracy: ", round(accuracy, digits=2), " by ", string(pipeline))
        catch
            continue
        end
    end
    return (best_accuracy, best_pipeline)
end

In [ ]:
find_best_pipeline_with_vlsn(grammar, train_X, train_Y, test_X, test_Y, 2)

In [ ]:
function p_cost_f(pipeline)
    # pipeline = eval(pipeline)
    return pipeline_cost_function(pipeline, train_X, train_Y, test_X, test_Y)
end

enumerator = Herb.HerbSearch.get_vlsn_enumerator(grammar, [], 2, :START, p_cost_f)
# Herb.HerbSearch.get_mh_enumerator()
c = 0
for rule in enumerator
    println(Herb.HerbSearch.rulenode2expr(rule, grammar))
    c = c + 1
    if c == 20
        break
    end
end